In [44]:
import pandas as pd
from dateutil import parser
from itertools import chain, groupby
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score,confusion_matrix
import math
from collections import namedtuple
from itertools import chain, groupby, combinations
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import psycopg2
import pymc3 as pm
import pymc3 as pm, theano.tensor as tt
from sqlalchemy import create_engine
import io
from sklearn.neural_network import MLPClassifier

In [2]:
rds_endpoint = 'creditrisk.cnhiln9arozo.eu-west-1.rds.amazonaws.com'
rds_user = 'lukhanyo2'
rds_psswrd = 'Popart221'
port = 5432
rds_database = 'creditdb'

In [3]:
cd data

C:\Users\lukha\Google Drive\credit_risk\data


In [4]:
pd.options.mode.chained_assignment = None 
try:
    conn = psycopg2.connect("dbname='{:s}' user='{:s}' host='{:s}' password='{:s}'".format(rds_database, rds_user, rds_endpoint, rds_psswrd))
    print("Connected to the database")
except:
    print("Unable to connect to the database, please check your settings")

Connected to the database


In [5]:
application_train = pd.read_sql_query("select * from application_train", con=conn)

In [6]:
application_test = pd.read_sql_query("select * from application_test", con=conn)

In [7]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,138139,0,Cash loans,M,Y,Y,0,362250.0,555273.0,17910.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
1,138140,0,Cash loans,M,Y,Y,0,225000.0,225000.0,14778.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0
2,138141,0,Cash loans,M,N,Y,0,90000.0,536917.5,17874.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
3,138142,1,Cash loans,M,N,Y,0,117000.0,222547.5,15957.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0
4,138143,0,Cash loans,M,Y,Y,1,180000.0,526491.0,26743.5,...,0,0,0,0,0.0,0.0,0.0,0.0,2.0,1.0


In [8]:
#mf = pd.get_dummies(application_train['CODE_GENDER']).head().drop('F',axis = 1)
#pd.concat([mf.head(),application_train.head(10)],axis = 1)

In [9]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,138139,0,Cash loans,M,Y,Y,0,362250.0,555273.0,17910.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
1,138140,0,Cash loans,M,Y,Y,0,225000.0,225000.0,14778.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0
2,138141,0,Cash loans,M,N,Y,0,90000.0,536917.5,17874.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
3,138142,1,Cash loans,M,N,Y,0,117000.0,222547.5,15957.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0
4,138143,0,Cash loans,M,Y,Y,1,180000.0,526491.0,26743.5,...,0,0,0,0,0.0,0.0,0.0,0.0,2.0,1.0


In [10]:
application_train[application_train['CODE_GENDER'] == 'XNA'].count()

SK_ID_CURR                    4
TARGET                        4
NAME_CONTRACT_TYPE            4
CODE_GENDER                   4
FLAG_OWN_CAR                  4
FLAG_OWN_REALTY               4
CNT_CHILDREN                  4
AMT_INCOME_TOTAL              4
AMT_CREDIT                    4
AMT_ANNUITY                   4
AMT_GOODS_PRICE               4
NAME_TYPE_SUITE               4
NAME_INCOME_TYPE              4
NAME_EDUCATION_TYPE           4
NAME_FAMILY_STATUS            4
NAME_HOUSING_TYPE             4
REGION_POPULATION_RELATIVE    4
DAYS_BIRTH                    4
DAYS_EMPLOYED                 4
DAYS_REGISTRATION             4
DAYS_ID_PUBLISH               4
OWN_CAR_AGE                   2
FLAG_MOBIL                    4
FLAG_EMP_PHONE                4
FLAG_WORK_PHONE               4
FLAG_CONT_MOBILE              4
FLAG_PHONE                    4
FLAG_EMAIL                    4
OCCUPATION_TYPE               2
CNT_FAM_MEMBERS               4
                             ..
DEF_30_C

In [11]:
pd.get_dummies(application_train['CODE_GENDER'],).head()#.drop('N',axis = 1)

,F,M,XNA
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0


In [12]:
#pd.get_dummies(application_test['CODE_GENDER'])['M']
gender = pd.get_dummies(application_test['CODE_GENDER'])['M']#.drop(['F','XNA'],axis = 1) # CODE_GENDER
gender.columns = ['Male']

In [13]:
def preprocessing(df):
    gender = pd.get_dummies(df['CODE_GENDER'])['M']#.drop(['F','XNA'],axis = 1) # CODE_GENDER

    #df.drop(['CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY'],axis = 1,inplace = True)
    owncar = pd.get_dummies(df['FLAG_OWN_CAR']).drop('N',axis = 1)
    owncar.columns = ['OWN_CAR_Y']
    
    ownreal = pd.get_dummies(df['FLAG_OWN_REALTY']).drop('N',axis = 1)
    ownreal.columns = ['OWN_REALTY_Y']
    try:
        incometype = pd.get_dummies(df['NAME_INCOME_TYPE']).drop('Maternity leave',axis = 1)
    except:
        incometype = pd.get_dummies(df['NAME_INCOME_TYPE'])
        
    educationtype = pd.get_dummies(df['NAME_EDUCATION_TYPE'])#.drop('N',axis = 1)
    df2 = pd.concat([gender,owncar,ownreal],axis = 1)
    
    df2.columns = ['Male','OWN_CAR_Y','OWN_REALTY_Y']
    
    df3 = pd.concat([df2,incometype,educationtype],axis = 1)
    df = df.drop(['CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE'],axis = 1)
    df = pd.concat([df,df3],axis =1)
    return df

In [14]:
#bureau = pd.read_sql_query('select * from bureau join bureau_balance using ("SK_ID_BUREAU") limit 10', con=conn)

In [15]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,138139,0,Cash loans,M,Y,Y,0,362250.0,555273.0,17910.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
1,138140,0,Cash loans,M,Y,Y,0,225000.0,225000.0,14778.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0
2,138141,0,Cash loans,M,N,Y,0,90000.0,536917.5,17874.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
3,138142,1,Cash loans,M,N,Y,0,117000.0,222547.5,15957.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0
4,138143,0,Cash loans,M,Y,Y,1,180000.0,526491.0,26743.5,...,0,0,0,0,0.0,0.0,0.0,0.0,2.0,1.0


In [16]:
clean_application_train = preprocessing(application_train)
clean_application_train.dropna(inplace = True)
clean_application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_FAMILY_STATUS,...,Pensioner,State servant,Student,Unemployed,Working,Academic degree,Higher education,Incomplete higher,Lower secondary,Secondary / secondary special
45,138188,0,Cash loans,0,202500.0,675000.0,21775.5,675000.0,Unaccompanied,Married,...,0,0,0,0,0,0,0,0,0,1
100,138251,0,Cash loans,0,90000.0,526491.0,26878.5,454500.0,Unaccompanied,Married,...,0,0,0,0,1,0,1,0,0,0
115,138265,1,Cash loans,0,270000.0,640080.0,29970.0,450000.0,Unaccompanied,Single / not married,...,0,0,0,0,0,0,1,0,0,0
184,138349,0,Cash loans,1,337500.0,582768.0,31176.0,540000.0,Unaccompanied,Widow,...,0,1,0,0,0,0,0,0,0,1
221,138387,0,Cash loans,0,90000.0,391500.0,14886.0,391500.0,Unaccompanied,Married,...,0,0,0,0,1,0,0,0,0,1


In [17]:
#bureau[bureau['CREDIT_ACTIVE'] == 'Active']

In [18]:
application_train.dropna(inplace = True)
y1 = clean_application_train['TARGET']
X1 = clean_application_train.drop(['SK_ID_CURR','TARGET','EMERGENCYSTATE_MODE','WALLSMATERIAL_MODE','HOUSETYPE_MODE','FONDKAPREMONT_MODE','ORGANIZATION_TYPE',\
                            'OCCUPATION_TYPE','WEEKDAY_APPR_PROCESS_START','NAME_HOUSING_TYPE','NAME_FAMILY_STATUS','NAME_CONTRACT_TYPE'\
                           ,'NAME_TYPE_SUITE','REGION_POPULATION_RELATIVE'],axis = 1)


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.33, random_state=42)

In [45]:
model = MLPClassifier()

In [46]:
model.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [47]:
model.predict(X_test)

array(['0', '0', '0', ..., '0', '0', '0'], dtype='<U1')

In [58]:
1-len(clean_application_train[clean_application_train['TARGET']=='1'])/len(clean_application_train)

0.9388514299000232

In [60]:
X_test.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,...,Pensioner,State servant,Student,Unemployed,Working,Academic degree,Higher education,Incomplete higher,Lower secondary,Secondary / secondary special
209754,1,315000.0,971280.0,62203.5,900000.0,-17041,-1204,-1194.0,-588,13.0,...,0,0,0,0,0,0,0,0,0,1
153326,0,112500.0,270000.0,13500.0,270000.0,-14823,-1463,-3115.0,-5626,17.0,...,0,0,0,0,1,0,0,0,0,1
219257,1,135000.0,422892.0,25870.5,382500.0,-8567,-196,-2325.0,-890,64.0,...,0,0,0,0,0,0,0,0,0,1
296209,0,90000.0,254700.0,26874.0,225000.0,-18440,-1012,-9521.0,-1977,12.0,...,0,0,0,0,1,0,0,0,0,1
198681,1,112500.0,679500.0,24201.0,679500.0,-15120,-472,-2689.0,-3996,7.0,...,0,0,0,0,1,0,1,0,0,0


In [61]:
confusion_matrix(y_test,model.predict(X_test))

array([[2655,    6],
       [ 178,    0]], dtype=int64)

In [26]:
model.score(X_test,y_test)

0.9369496301514618

In [28]:
aa = list(application_test.columns)

In [29]:
aa.sort()

In [30]:
aa

['AMT_ANNUITY',
 'AMT_CREDIT',
 'AMT_GOODS_PRICE',
 'AMT_INCOME_TOTAL',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_YEAR',
 'APARTMENTS_AVG',
 'APARTMENTS_MEDI',
 'APARTMENTS_MODE',
 'BASEMENTAREA_AVG',
 'BASEMENTAREA_MEDI',
 'BASEMENTAREA_MODE',
 'CNT_CHILDREN',
 'CNT_FAM_MEMBERS',
 'CODE_GENDER',
 'COMMONAREA_AVG',
 'COMMONAREA_MEDI',
 'COMMONAREA_MODE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_ID_PUBLISH',
 'DAYS_LAST_PHONE_CHANGE',
 'DAYS_REGISTRATION',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'ELEVATORS_AVG',
 'ELEVATORS_MEDI',
 'ELEVATORS_MODE',
 'EMERGENCYSTATE_MODE',
 'ENTRANCES_AVG',
 'ENTRANCES_MEDI',
 'ENTRANCES_MODE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'FLAG_CONT_MOBILE',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT

In [31]:
application_test = pd.read_sql_query("select * from application_test", con=conn)

In [67]:
application_test.dtypes.value_counts()

object    121
dtype: int64

In [32]:
application_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,None,None,None,None,None,None


In [33]:
clean_application_test.head()

NameError: name 'clean_application_test' is not defined

In [34]:
#application_test = pd.read_sql_query("select * from application_test", con=conn)
clean_application_test = preprocessing(application_test)
clean_application_test.fillna('1',inplace = True)

X = clean_application_test.drop(['SK_ID_CURR','EMERGENCYSTATE_MODE','WALLSMATERIAL_MODE','HOUSETYPE_MODE','FONDKAPREMONT_MODE','ORGANIZATION_TYPE',\
                            'OCCUPATION_TYPE','WEEKDAY_APPR_PROCESS_START','NAME_HOUSING_TYPE','NAME_FAMILY_STATUS','NAME_CONTRACT_TYPE'\
                           ,'NAME_TYPE_SUITE','REGION_POPULATION_RELATIVE'],axis = 1)

In [35]:
#X['Male']

In [36]:
preds = model.predict(X)

In [37]:
confusion_matrix(y_test,model.predict(X_test))

48744

In [38]:
ans = pd.concat([pd.DataFrame(application_test['SK_ID_CURR']),pd.DataFrame(preds,columns=['TARGET'])],ignore_index=True,axis=1)
ans.columns = ['SK_ID_CURR','TARGET']

In [39]:
ans.to_csv('answer.csv',index = False)

In [62]:
!cd

C:\Users\lukha\Google Drive\credit_risk\data


In [ ]:
list(X1.columns)

In [ ]:
list(X.columns)

In [ ]:
set(list(X1.columns)) - set(list(X.columns))

In [ ]:
ans.tail()

C:\Users\lukha\Google Drive\credit_risk
